In [1]:
from keras.layers import Dense, Activation, Conv2D
from keras.layers import MaxPooling2D, Dropout, Flatten,GlobalMaxPooling2D
from keras.models import Sequential
import random
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
# from keras.layers import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

2023-02-10 15:00:16.774379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 15:00:16.918312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 15:00:16.918345: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 15:00:17.740710: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def build_model(layer_dims, input_shape=(256,256,3,),len_classes=3, dropout_rate=0.2,activation='relu'):
    print(1)
    """Function to build a model with specified layer dimensions and activation function."""
    model = Sequential()
    for i, dim in enumerate(layer_dims):
        if i == 0:
            model.add(Conv2D(dim[0], dim[1], input_shape=input_shape, activation=activation))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        else:
            model.add(Conv2D(dim[0], dim[1], activation=activation))
            model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=len_classes, kernel_size=1, strides=1))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling2D())
    model.add(Activation('softmax'))
    return model

In [3]:
def sample_architecture(min_layers=1, max_layers=5, min_filters=32, max_filters=512, min_kernel=3, max_kernel=5):
    """Function to sample a random architecture from the search space."""
    num_layers = random.randint(min_layers, max_layers)
    layer_dims = [(random.randint(min_filters, max_filters), random.randint(min_kernel, max_kernel)) for _ in range(num_layers)]
    return layer_dims

In [4]:
def mutate_architecture(layer_dims, mutation_rate=0.1, min_layers=1, max_layers=5, min_filters=32, max_filters=512, min_kernel=3, max_kernel=5):
    """Function to mutate an architecture by randomly modifying some of its layer dimensions."""
    num_layers = len(layer_dims)
    for i in range(num_layers):
        if np.random.rand() < mutation_rate:
            layer_dims[i] = (random.randint(min_filters, max_filters), random.randint(min_kernel, max_kernel))
    return layer_dims

In [5]:
# def breed_architectures(parent1, parent2, mutation_rate=0.1, min_layers=1, max_layers=5, min_filters=32, max_filters=512, min_kernel=3, max_kernel=5):
#     """Function to breed two architectures to produce a new architecture."""
#     child = []
#     for i in range(len(parent1)):
#         if np.random.rand() < 0.5:
#             child.append(parent1[i])
#         else:
#             child.append(parent2[i])
#     return mutate_architecture(child, mutation_rate, min_layers, max_layers, \
#                                min_filters, max_filters, min_kernel, max_kernel)


In [6]:
def breed_architectures(parent1, parent2, mutation_rate, min_layers, max_layers, min_filters, max_filters, min_kernel, max_kernel):
    """Function to breed two parent architectures to produce a child architecture."""
    child = []
    for i in range(len(parent1)):
        if np.random.rand() < 0.5:
            child.append(parent1[i])
        else:
            if i < len(parent2):
                child.append(parent2[i])
    return mutate_architecture(child, mutation_rate, min_layers, max_layers, min_filters, max_filters, min_kernel, max_kernel)

In [7]:
def get_data(train_dir,validation_dir,batch_size=60,test_train_split=0.2):
    train_data = image_dataset_from_directory(\
      train_dir,color_mode="grayscale",image_size=(256,256) ,\
      subset='training',seed=12, validation_split=test_train_split,\
      batch_size=batch_size)
    validation_data = image_dataset_from_directory(validation_dir,
      color_mode="grayscale",image_size=(256,256), subset='validation',seed=12,\
      validation_split=test_train_split,batch_size=batch_size)
    return train_data,validation_data

In [8]:
#Train the model on the dataset and evaluate its performance.
def train_and_evaluate_model(model,epochs=10, train_dir=None,X_train=None, y_train=None,\
                             X_test=None,y_test=None):
    """Function to train and evaluate a model."""
    if train_dir is not None:
        train_data,validation_data=get_data(train_dir,train_dir)
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        history = model.fit(train_data, epochs=epochs, verbose=1,validation_data=validation_data)
        # return history
        # Extract the accuracy from the history object
        acc = history.history['val_accuracy'][len(history.history['val_accuracy'])-1]
        return acc
    else:
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)
        scores = model.evaluate(X_test, y_test, verbose=1)
        return scores[1]  # Return accuracy

In [9]:
# 4) Define the genetic algorithm to evolve the architecture.
def genetic_algorithm(train_dir,epochs,population_size=20,len_classes=3, num_generations=10, mutation_rate=0.1,\
                      min_layers=1, max_layers=5, min_filters=32, max_filters=512,\
                      min_kernel=3, max_kernel=5):
    """Function to implement the genetic algorithm to evolve the architecture."""
    # Initialize population
    population = [sample_architecture(min_layers, max_layers, min_filters,\
                                      max_filters,min_kernel, max_kernel)\
                  for _ in range(population_size)]
    scores = []
    for i in range(num_generations):
        print("Generation {}/{}".format(i + 1, num_generations))
        new_population = []
        for j in range(population_size):
            # Select two random parents
            the_choice=np.random.choice(population_size, 2, replace=False)
            parent1=population[the_choice[0]]
            parent2=population[the_choice[1]]
            # Breed the parents to produce a child
            child = breed_architectures(parent1, parent2, mutation_rate,\
                                        min_layers,max_layers, min_filters,\
                                        max_filters, min_kernel, max_kernel)
            new_population.append(child)
        # Evaluate each model in the population
        scores = []
        for j, layer_dims in enumerate(new_population):
            model = build_model(layer_dims,len_classes=len_classes,input_shape=(256,256,1,))
            # (X_train, y_train), (X_test, y_test)=tf.keras.datasets.mnist.load_data()
            score = train_and_evaluate_model(model,epochs=epochs,train_dir=train_dir)
#  X_train, y_train, X_test, y_test)
            scores.append((layer_dims, score))
        # Sort the models by accuracy and select the best ones for the next generation
        scores.sort(key=lambda x: x[1], reverse=True)
        population = [x[0] for x in scores[:population_size]]
    # Return the best architecture
    return population[0]

In [10]:
train_dir="/home/shubhransu_etc/project_sys22/data/mammogram"
test_dir="/content/drive/MyDrive/cbis_ddsm/mass/test"

In [ ]:
# input_shape = (200, 100,3) 
# Run the genetic algorithm to evolve the architecture
best_architecture = genetic_algorithm(train_dir=train_dir,epochs=5)

Generation 1/10
1


2023-02-10 15:00:21.038602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-10 15:00:21.038653: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-10 15:00:21.038691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (paramshavak-CELSIUS-R940): /proc/driver/nvidia/version does not exist
2023-02-10 15:00:21.039166: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 552 files belonging to 3 classes.
Using 442 files for training.
Found 552 files belonging to 3 classes.
Using 110 files for validation.
Epoch 1/5


2023-02-10 15:00:34.217014: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 96 of 480
2023-02-10 15:00:44.914096: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 226 of 480
2023-02-10 15:00:54.359293: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 345 of 480
2023-02-10 15:01:01.558874: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


8/8 [==============================] - 178s 18s/step - loss: 0.9661 - accuracy: 0.4344 - val_loss: 1.5560 - val_accuracy: 0.4000
Epoch 2/5
8/8 [==============================] - 140s 16s/step - loss: 0.8952 - accuracy: 0.4683 - val_loss: 1.1463 - val_accuracy: 0.4545
Epoch 3/5
8/8 [==============================] - 146s 17s/step - loss: 0.8944 - accuracy: 0.4774 - val_loss: 1.0164 - val_accuracy: 0.5727
Epoch 4/5
8/8 [==============================] - 143s 17s/step - loss: 0.8931 - accuracy: 0.4932 - val_loss: 0.9552 - val_accuracy: 0.5091
Epoch 5/5


...

1/8 [==>...........................] - ETA: 4:32 - loss: 1.4266 - accuracy: 0.2167

2023-02-10 15:14:00.780583: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 7432243200 exceeds 10% of free system memory.
2023-02-10 15:14:21.317899: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 7432243200 exceeds 10% of free system memory.


2/8 [======>.......................] - ETA: 2:55 - loss: 1.2454 - accuracy: 0.3417

2023-02-10 15:14:30.016214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 7432243200 exceeds 10% of free system memory.


8/8 [==============================] - 243s 29s/step - loss: 1.0815 - accuracy: 0.4118 - val_loss: 3.9185 - val_accuracy: 0.3909
Epoch 2/5
4/8 [==============>...............] - ETA: 2:06 - loss: 0.9978 - accuracy: 0.4333

In [ ]:
best_architecture

In [ ]:
# def build_model(input_shape=(224,224,3,),len_classes=3, dropout_rate=0.2,activation='relu'):
#     """Function to build a model with specified layer dimensions and activation function."""
#     model = Sequential()
#     model.add(Conv2D(64, 3, input_shape=input_shape, activation=activation))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Conv2D(128, 3, activation=activation))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(filters=len_classes, kernel_size=1, strides=1))
#     model.add(Dropout(dropout_rate))
#     model.add(BatchNormalization())
#     model.add(GlobalMaxPooling2D())
#     model.add(Activation('softmax'))
#     return model

In [ ]:
# model=build_model(len_classes=3,input_shape=(256,256,1,))

In [ ]:
# his=train_and_evaluate_model(model,epochs=2,train_dir=train_dir)

In [ ]:
# his.history['val_accuracy'][len(his.history['val_accuracy'])-1]

In [ ]:
# dir(keras.layers.sequential().fit)